# 1. Import Data

In [1]:
from datetime import datetime, timedelta
import re
import pandas as pd
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = '/content/drive/MyDrive/[MVC]공동연구개발/네이버뉴스수집'

In [ ]:
# 파일 수정은 여기서
# article_df = pd.read_excel('/content/drive/MyDrive/Task/MVC/date/data/조매연_2.xlsx')
# article_df = pd.read_excel('/content/drive/MyDrive/Task/MVC/date/data/TEST.xlsx')

# 언론사 수정 본
article_df = pd.read_excel(base_path + '/네이버_언론사별_최신_경제_뉴스_2024-05-22.xlsx')

In [ ]:
article_df['생성일자'] = pd.to_datetime(article_df['생성일자'])
# article_df['생성일자'].astype(str)

article_df.head()
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   기사제목    60 non-null     object        
 1   생성일자    60 non-null     datetime64[ns]
 2   세부분류    60 non-null     object        
 3   언론사     60 non-null     object        
 4   url     60 non-null     object        
 5   기사_내용   60 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 2.9+ KB


# 2. Preprocessing

## 2-0. 날짜 치환 함수 정의

In [ ]:
def process_dates_in_article(article_date, text):
    # Helper function to find the next Friday
    def find_friday(date):
        current_weekday = date.weekday()
        days_until_friday = 4 - current_weekday
        return date + timedelta(days=days_until_friday)

    # Helper function to adjust the date for "지난 n일" pattern
    def adjust_date(year, month, day):
        if month == 1:
            return datetime(year - 1, 12, day)
        else:
            return datetime(year, month - 1, day)

    # Define the regular expressions and corresponding lambda functions
    regex_patterns = {
        # DAY
        "어제": ("어제",
                        lambda match: (article_date - timedelta(days=1)).strftime('%Y-%m-%d')),
        "오늘": ("오늘|금일|이[- ]?날", # 5월 18일
                        lambda match: article_date.strftime('%Y-%m-%d')),
        "내일": ("내일",
                        lambda match: (article_date + timedelta(days=1)).strftime('%Y-%m-%d')),
        # 지난 n~m일 5월 19일
        "지난 n~m일": ("지난[- ]?(\d+)\u223C(\d+)일",
                        lambda match: (adjust_date(article_date.year, article_date.month, int(match.group(1))) if int(match.group(1)) > article_date.day else article_date.replace(day=int(match.group(1)))).strftime('%Y-%m-%d') +
                                      "~" +
                                      (adjust_date(article_date.year, article_date.month, int(match.group(2))) if int(match.group(2)) > article_date.day else article_date.replace(day=int(match.group(2)))).strftime('%Y-%m-%d')),
        "지난 n일": ("지난[- ]?(\d+)일",
                        lambda match: (adjust_date(article_date.year, article_date.month, int(match.group(1))) if int(match.group(1)) > article_date.day else article_date.replace(day=int(match.group(1)))).strftime('%Y-%m-%d')),
        "지난달 n일": ("지난[- ]?달[- ]?(\d+)일|이전[- ]?달[- ]?(\d+)일|전[- ]?달[- ]?(\d+)일|저번[- ]?달[- ]?(\d+)일|전[- ]?월[- ]?(\d+)일", # 5월 18일
                        lambda match: adjust_date(article_date.year, article_date.month, int(match.group(1) or match.group(2) or match.group(3) or match.group(4) or match.group(5))).strftime('%Y-%m-%d')),
        "이달 n~m일": ("이달[- ]?(\d+)\u223C(\d+)일|이번달[- ]?(\d+)\u223C(\d+)일|금월[- ]?(\d+)\u223C(\d+)일|", # 5월 18일
                        lambda match: (datetime(article_date.year, article_date.month, int(match.group(1) or match.group(3) or match.group(5))).strftime('%Y-%m-%d') +
                                       "~" +
                                       datetime(article_date.year, article_date.month, int(match.group(2) or match.group(4) or match.group(6))).strftime('%Y-%m-%d'))
                                        if (match.group(1) or match.group(3) or match.group(5)) and (match.group(2) or match.group(4) or match.group(6))
                                        else match.group(0)),
        "이달 n일": ("이달[- ]?(\d+)일|이번[- ]?달[- ]?(\d+)일|금월[- ]?(\d+)일", # 5월 18일
                        lambda match: article_date.replace(day=int(match.group(1) or match.group(2) or match.group(3))).strftime('%Y-%m-%d')),
          # 5월 18일
        "다음달 n~m일": ("내달[- ]?(\d+)\u223C(\d+)일|다음[- ]?달[- ]?(\d+)\u223C(\d+)일",
                        lambda match: (datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1) or match.group(3))).strftime('%Y-%m-%d')) +
                                       "~" +
                                       (datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(2) or match.group(4))).strftime('%Y-%m-%d'))
                                        if (match.group(1) or match.group(3)) and (match.group(2) or match.group(4))
                                        else match.group(0)
                                        ),
        "다음달 n일": ("내달[- ]?(\d+)일|다음[- ]?달[- ]?(\d+)일",
                        lambda match: datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1) or match.group(2))).strftime('%Y-%m-%d')),
        "오는 n일": ("오는[- ]?(\d+)일",
                        lambda match: (article_date.replace(day=int(match.group(1))) if int(match.group(1)) > article_date.day else datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1)))).strftime('%Y-%m-%d')),
        "오는 n~m일": ("오는[- ]?(\d+)\u223C(\d+)일",
                        lambda match: (article_date.replace(day=int(match.group(1))) if int(match.group(1)) > article_date.day else datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(1)))).strftime('%Y-%m-%d') +
                                      "~" +
                                      (article_date.replace(day=int(match.group(2))) if int(match.group(2)) > article_date.day else datetime(article_date.year + 1 if article_date.month == 12 else article_date.year, 1 if article_date.month == 12 else article_date.month + 1, int(match.group(2)))).strftime('%Y-%m-%d')),
        "올해 n월 n일": ("올해[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "내년 n월 n일": ("내년[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year + 1, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
        "작년 n월 n일": ("작년[- ]?(\d+)월[- ]?(\d+)일",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),
          # 5월 18일
        # "n월 n일": ("(\d+)월[- ]?(\d+)일",
        "n월 n일": ("(?<!\d{4}년 )(?<!매년 )(\d+)월[- ]?(\d+)일", # '매년 n월 m일' 안 잡히게 수정
                        lambda match: datetime(article_date.year, int(match.group(1)), int(match.group(2))).strftime('%Y-%m-%d')),

        # 5월 19일
        # n~m일 (예정)
        "n일": ("([1-9]|1[0-9]|2[0-9]|3[01])일",
                        lambda match: (datetime(article_date.year - 1 if article_date.month == 1 and int(match.group(1)) > article_date.day else article_date.year,
                                                12 if article_date.month == 1 and int(match.group(1)) > article_date.day else article_date.month - 1 if int(match.group(1)) > article_date.day else article_date.month,
                                                int(match.group(1)))).strftime('%Y-%m-%d')),

        # WEEK
        "이번주": ("이번[- ]?주(?!말)", # 5/19 수정
                        lambda match: find_friday(article_date).strftime('%Y-%m')),
        "다음주": ("다음[- ]?주(?!말)", # 차주는 추가했다가 사고가 날 거 같다
                        lambda match: (find_friday(article_date) + timedelta(days=7)).strftime('%Y-%m')),
        "지난주": ("지난[- ]?주(?!말)",
                        lambda match: (find_friday(article_date) - timedelta(days=7)).strftime('%Y-%m')),


        # MONTH
        "작년 n월": ("작년[- ]?(\d+)월",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), 1).strftime('%Y-%m')),
        "지난해 n월": ("지난[- ]?해 (\d+)월",
                        lambda match: datetime(article_date.year - 1, int(match.group(1)), 1).strftime('%Y-%m')),
        "지난 n월": ("지난[- ]?(\d+)월",
                        lambda match: datetime(article_date.year - 1 if article_date.month < int(match.group(1)) else article_date.year, int(match.group(1)), 1).strftime('%Y-%m')),
          # 5월 18일
        # "지난달": ("지난[- ]?달|이전[- ]?달|전[- ]?달|저번[- ]?달|전[- ]?월",
        "지난달": ("지난[- ]?달|이전[- ]?달|저번[- ]?달|전[- ]?월", # '전달' 삭제 (5/19)
                        lambda match: (article_date.replace(day=1) - timedelta(days=1)).replace(day=1).strftime('%Y-%m')),
        "이달": ("이달|이번[- ]?달|금월",
                        lambda match: article_date.strftime('%Y-%m')),
        "다음달": ("다음[- ]?달|내달(?![가-힣])|차월|익월", # '내달'이 단독으로 쓰이도록 수정 (5/19)
                        lambda match: (article_date.replace(day=28) + timedelta(days=4)).replace(day=1).strftime('%Y-%m')),

        # 분기(Q)
        "YYYY년 N분기": ("(\d{4})년[- ]?(\d{1})분기",
                        lambda match: match.group(1) + '-' + match.group(2) + 'Q'),
        "지난해 N분기": ("지난해[- ]?(\d{1})분기",
                        lambda match: str(article_date.year - 1) + '-' + match.group(1) + 'Q'),
        "올해 N분기": ("올해[- ]?(\d{1})분기",
                        lambda match: str(article_date.year) + '-' + match.group(1) + 'Q'),
        "올 N분기": ("올[- ]?(\d{1})분기",
                        lambda match: str(article_date.year) + '-' + match.group(1) + 'Q'),

        # 상반기(1H)
        "YYYY년 상반기": ("(\d{4})년[- ]?상반기",
                        lambda match: match.group(1) + '-1H'),
        "지난해 상반기": ("지난해[- ]?상반기",
                        lambda match: str(article_date.year - 1) + '-1H'),
        "올해 상반기": ("올해[- ]?상반기",
                        lambda match: str(article_date.year) + '-1H'),
        "올 상반기": ("올[- ]?상반기",
                        lambda match: str(article_date.year) + '-1H'),
        # 하반기(2H)
        "YYYY년 하반기": ("(\d{4})년[- ]?하반기",
                        lambda match: match.group(1) + '-2H'),
        "지난해 하반기": ("지난해[- ]?하반기",
                        lambda match: str(article_date.year - 1) + '-2H'),
        "올해 하반기": ("올해[- ]?하반기",
                        lambda match: str(article_date.year) + '-2H'),
        "올 하반기": ("올[- ]?하반기",
                        lambda match: str(article_date.year) + '-2H'),

        # YEAR
        "올해": ("올해(?!의)", # '올해의' 예외 처리 5/19
                        lambda match: article_date.strftime('%Y년')),
        "작년": ("작년",
                        lambda match: datetime(article_date.year - 1, 1, 1).strftime('%Y년')),
        "지난해": ("지난[- ]?해",
                        lambda match: datetime(article_date.year - 1, 1, 1).strftime('%Y년')),
        "내년": ("내년",
                        lambda match: datetime(article_date.year + 1, 1, 1).strftime('%Y년'))
    }

    # Replace patterns in the text with the calculated date strings
    for key, (pattern_str, replacement_func) in regex_patterns.items():
        pattern = re.compile(pattern_str)
        # For "지난 n일", special handling to adjust the month correctly
        if key == "지난 n일":
            def replacement(match):
                day = int(match.group(1))
                if day > article_date.day:
                    new_date = adjust_date(article_date.year, article_date.month, day)
                    return new_date.strftime('%Y-%m-%d')
                else:
                    return article_date.replace(day=day).strftime('%Y-%m-%d')
            text = pattern.sub(replacement, text)
        else:
            text = pattern.sub(replacement_func, text)

    # Helper function to reformat date strings
    def reformat_dates(text):
        pattern = r'\d{4}-\d{2}(?:-\d{2})?'

        def format_match(match):
            date_str = match.group(0)
            try:
                if '-' in date_str[7:]:  # Looks for the second '-' in 'YYYY-MM-DD'
                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    return date_obj.strftime('%Y년 %m월 %d일')
                else:
                    date_obj = datetime.strptime(date_str, '%Y-%m')
                    return date_obj.strftime('%Y년 %m월')
            except ValueError:
                return date_str

        return re.sub(pattern, format_match, text)

    # Reformat dates in the processed text
    formatted_text = reformat_dates(text)

    return formatted_text

# # Example usage
# article_date = datetime(2023, 5, 6)
# text = "이날 어제 오늘 지난 5일 지난달 3일 지난주 이번주 다음주 올해 5월 1일"
# formatted_text = process_dates_in_article(article_date, text)
# print(formatted_text)


## 2-1. apply 적용을 위한 함수 정의

In [ ]:
def apply_process_dates(row):
    return process_dates_in_article(row['생성일자'], row['기사_내용'])

## 2-2 article_df 즉, xlsx 파일 input

In [ ]:
article_df['기사_내용_date_replacement'] = article_df.apply(apply_process_dates, axis=1)

# article_df['기사_내용_date_replacement'].loc[6]

## 2-3. `기사제목`, `생성일자`, `언론사`와 concat해서 새로운 열 생성
- GPT에 Prompting 편의를 위함

#### 문장 쪼개기

In [ ]:
prompt_top_split = '''Task: Rewrite each sentence of the article to be clear and concise. Maintain the existence and accuracy of dates and numerical information, and ensure no details are omitted.

Instructions:

1. Preserve Meaning: Each rewritten sentence must convey the same information as the original, especially maintaining the accuracy of dates and numerical data. Never forget to keep the numerical data intact.
2. Separate Connected Sentences: Break down complex sentences connected with "~고" or "~며" into individual sentences centered around a single verb.
3. Avoid Redundancy: Prevent repetition of the same information.
4. Keep the article title, publication date, and media information as is.
5. If it is completely unrelated to the investment decision for the company, it can be omitted. (However, conservatively)
6. Answer in Korean.

Final Review: Examine the text to ensure no sentences could be simplified further or split into smaller segments. Make necessary adjustments to achieve the utmost clarity and brevity.

기사 시작
```'''

prompt_bottom_split = '''```
기사 끝'''


In [ ]:
article_df['GPT_input_text_split'] = prompt_top_split + "\n" + "기사 제목: " + article_df['기사제목'].map(str) + "\n" + "기사 발행일자: " + article_df['생성일자'].map(str) + "\n" + "언론사: " + article_df['언론사'].map(str) + "\n" + "기사 내용: " + article_df['기사_내용_date_replacement'].map(str) + "\n" + prompt_bottom_split

# Export Data

In [ ]:
article_df['GPT_output_text_split'] = ''

### 말짱한 Export

### 아래는 실험할 때 Export

In [ ]:
# pip install pytz
import pytz

In [ ]:
now = (datetime.now(pytz.timezone('Asia/Seoul')).strftime("%m_%d_%H:%M"))
article_df.to_excel('/content/drive/MyDrive/Task/MVC/date/output/' + now + '.xlsx')

# 문장 쪼갠 다음

In [16]:
# 파일 수정은 여기서
input_xlsx_name = '05_22_23:36'
article_df_splitted = pd.read_excel('/content/drive/MyDrive/Task/MVC/date/output/05_22_23:36.xlsx')

In [7]:
# pd.to_datetime(article_df['생성일자'])
# article_df['생성일자'].astype(str)

article_df_splitted.head()
article_df_splitted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              42 non-null     int64         
 1   기사제목                    42 non-null     object        
 2   생성일자                    42 non-null     datetime64[ns]
 3   세부분류                    42 non-null     object        
 4   언론사                     42 non-null     object        
 5   url                     42 non-null     object        
 6   기사_내용                   42 non-null     object        
 7   기사_내용_date_replacement  42 non-null     object        
 8   GPT_input_text_split    42 non-null     object        
 9   GPT_output_text_split   42 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 3.4+ KB


In [9]:
prompt_top = """You are a database administrator responsible for organizing essential information necessary for decision-making about buying and selling companies. Analyze the given articles and extract only the crucial information, summarizing it in JSON format. Please summarize only the information you need to know when deciding whether or not to buy a company from a private equity fund.

Think step by step:
1. Identify and summarize the most important content.
2. Organize the information into a maximum of three activities.
3. Eliminate duplicate content, prioritizing information that is frequently repeated as more important.

Answer in Korean.

Output 형태
{
    "제목": 기사의 제목
    "발행일시": 기사를 발행한 시간 형태는 YYYY-MM-DD HH24:MI:SS
    "언론사": 기사를 발행한 언론사명 출력. 파악하기 힘든 경우 null 출력.
    "내용": {
           [
            {
                "활동 1" : 주요 기업 활동 관련 키워드 1.  여기서 '주요 기업 활동'은 투자자가 꼭 알아야 하는 활동을 의미해.
                "활동 분류 1": "활동 1"에 가장 적합한 분류를 다음의 분류 중에 선택 -> 선택지: [구매], [판매], [수출], [수입], [전망], [실적], [추진], [출시], [기술], [투자], [자본], [인사], [노사], [고객], [사회], [환경], [안전], [규제], [지원], [법무], [협력]. 뒤의 기준을 참고해.

분류 기준 시작
```
매입하다, 구입하다, 구매하다, 도입하다-> [구매]
매출하다, 공급하다, 납품하다, 판매하다, 출하하다, 수주에 성공했다-> [판매]
수출하다 -> [수출]
수입하다 -> [수입]
시장 규모, 시장 전망, 성장률, 성장, 하강, 하락, 판매량 등에 대한 예상, 기대 -> [전망]
시장 규모, 시장 전망, 성장률, 성장, 하강, 하락, 판매량 등의 결과 -> [실적]
진출, 확장, 증축, 축소, 전략 등 → [추진]
생산, 제조, 출시, 발매, 서비스 출시 등 → [출시]
연구, 개발, 기술 확보, 시스템 구축, 특허 출원, 인증 등 → [기술]
투자, 합병, 인수, 매수, 출자, 회수, 펀드 결성, 엑시트 완료, 매각, 투자 유치, 투자 중단, 투자 종료 등 → [투자]
자본, 주식 소각, 증자, 감자, 배당, 대주주, 모회사, 계열사, 종속회사, 주주, 주식 발행 등 → [자본]
이사회, 경영진, 취임, 사임, 발령 등 → [인사]
노사 갈등, 교섭, 노사 협상 등 → [노사]
소비자 보호, 고객 상담, 리콜, 고객 응대, A/S, 고객서비스,고객 만족 등 → [고객]
사회 기여, 봉사, 지역 사회, 기부, 인권, 다양성, 사회적 책임 등 → [사회]
기후, 지속 가능성, 환경 등 → [환경]
사고, 재해 등 → [안전]
정부의 규제, 제재 → [규제]
정책 지원 등 정부의 지원이나 보조금 → [지원]
법무, 고소, 피고소, 고발, 사법리스크, 소송 등 -> [법무]
협력 관계, 협업 -> [협력]
```
분류 기준 끝

                "대상" : "활동 분류 1"을 기준으로, 그 대상이 되는  되는  제품, 서비스,  기술,  지적재산권,  사업내용  등을 출력. 예를 들어, [연구]이면 무엇을 연구했는지, [투자]이면 어디에 투자했는지, [전망]이면 무엇이 전망되는지를 출력한다.
                "활동 상세": "활동 1"의 기업 활동을 확인할 수 있는 실제 기사 문장을 원문 그대로 출력.
                "GPT 분류 1": Print the single word that best describes "활동 상세"
                "일자" :  "활동 상세"의 기업 활동이 발생한 연도, 반기, 분기, 월 등을 해당하는 뉴스 문장으로부터 확실히 파악할 수 있는 경우 작성한다. 해당 문장에서 활동의 발생일자를 파악할 수 없는 경우, null을 출력한다.
                "활동 주체" :  [
                         {"주체_기업명" : "활동 분류 1"을 기준으로, 활동의 주체인 기업 제시. 파악할 수 없는 경우 null.},
                         {"산업 분류" : "주체_기업명"에 들어가는 기업이 속한 산업군 제시. }]
                "활동 상대" :  [
                         {"상대_기업명" : "활동 분류 1"를 기준으로, 활동의 상대 또는 대상이 되는 기업 제시, 여러 기업일 경우 콤마(,)로 이어서 나열, 없으면 null},
                         {"산업 분류" : "상대_기업명"에 들어가는 기업이 속한 산업군 제시. }]
            "수치 요약": [
                {"수치": '활동 상세'에 출력된 문장에 등장한 "양(량)", "돈(금액,가격)", "율(비율,비중)"을 나타내는 구체적인 수치를 출력. 수치가 여러 개일 경우 콤마(,)로 나열. 구체적으로, 500대, 1000억, 500% 처럼 수치&단위만을 하나의 단어로 출력한다. 문장에 수치가 없는 경우 null. },
                {"수치_내용": 각 수치별로 respectively 수치의 의미를 설명(수치가 2개면 설명도 2개). 예를 들어, 500억 투자일 경우 '투자', 100억 손실일 경우 '손실'. 수치가 없는 경우 null."}
            ]
            },

            {
                "활동 2" : 주요 기업 활동 관련 키워드 2.  여기서 '주요 기업 활동'은 투자자가 꼭 알아야 하는 활동을 의미해.
                "GPT 분류 2": Print the single word that best describes "활동 2"
                "활동 분류 2": "활동 2"에 가장 적합한 분류를 다음의 분류 중에 선택 -> 선택지: [구매], [판매], [수출], [수입], [전망], [실적], [추진], [출시], [기술], [투자], [자본], [인사], [노사], [고객], [사회], [환경], [안전], [규제], [지원], [법무], [협력]
           // 추가 활동이 있을 경우 "활동 1"과 동일한 형식으로 반복

            },
        ]
    }
}


출력 결과 예시:
{
    "제목": 레인보우로보틱스, 사흘새 56%↑… 로봇주, 시장 주도하나
    "발행일자": 2023-03-17 16:32"
    "언론사": "오피니언뉴스"
    "내용": {
        [
            {
                "활동 1": 주식 장외 매수
                "활동 분류 1" : [투자]
                "대상물" : 보통주 91만3936주, 우선주 20만주
                "활동 상세": 삼성전자는 레인보우로보틱스 보통주 91만3936주와 우선주 20만주를 주당 3만400원에 장외 매수했다고 2023년 3월에 공시했다.
                "GPT 분류" : [매수]
                "일자" :  2023-03
                "활동 주체" :  [
                         {"주체_기업명" : 삼성전자},
                         {"산업 분류" : 전자 부품, 컴퓨터, 영상, 음향 및 통신장비 제조업}]
                "활동 상대" :  [
                         {"상대_기업명" : 레인보우로보틱스},
                         {"산업 분류" : 산업용 로봇 제조업}]

                "수치 요약": [
                    {"수치": 91만3936주, 20만주},
                    {"수치_내용": 보통주 장외 매수, 우선주 장외매수}

                ]

            },

            {
                "활동 2" : ....// 추가 활동이 있을 경우 "활동 1"과 동일한 형식으로 반복

-------------------------------------------------------------------------
아래부터가 실제 너가 분류할 기사 데이터이고, ```으로 기사의 시작과 끝을 표시해놨어.
출력 결과 예시는 반드시 제외하고 출력해.

기사 시작
```"""

prompt_bottom = "```\n기사 끝"

# print(prompt_bottom)

```
기사 끝


In [12]:
article_df_splitted['GPT_input_text_classification'] = prompt_top + "\n" + article_df_splitted['GPT_output_text_split'].map(str) + "\n" + prompt_bottom
article_df_splitted['GPT_output_text_classification'] = ''

In [21]:
output_xlsx_name = input_xlsx_name + '_add_prompt'
article_df_splitted.to_excel('/content/drive/MyDrive/Task/MVC/date/output/' + output_xlsx_name + '.xlsx')